# GardNamePreprocessor

In [ ]:
import pandas as pd
# Read the CSV file into a Pandas DataFrame
Gard = pd.read_csv('/content/exporttttt.csv')
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
import json
def extract_words_from_json_string(json_string):
    try:
        word_list = json.loads(json_string)
        words = [word.replace('"', '').strip() for word in word_list ]
        return words
    except (json.JSONDecodeError, TypeError) as e:
        #print(f"Error decoding JSON: {e}")
        return []

Gard['GardName'] = Gard['GardName'].apply(lambda x: str(x).replace('"', '').lower())
Gard['Synonyms'] = Gard['Synonyms'].apply(lambda x: extract_words_from_json_string(str(x).lower()))

import pandas as pd
def remove_similar_strings(df):
    for i in df.index:
        if i % 2000 ==0 : print(i)
        for j in df.index:
            if i != j:
                string_a = df['GardName'][i]
                list_b = df['Synonyms'][j]
                for item in list_b:  # Using [:] for iterating a copy of the list
                    if item == string_a:
                        list_b.remove(item)
    return df

Gard= remove_similar_strings(Gard)

In [39]:
import pandas as pd
import ast
def extract_words_from_json_string(input_string):
    try:
        # Use ast.literal_eval to safely convert the string to a list
        result_list = ast.literal_eval(input_string)
        if isinstance(result_list, list):
            return result_list
        else:
            raise ValueError("Input is not a string representation of a list.")
    except (ValueError, SyntaxError) as e:
        print(f"Error converting string to list: {e}")
        return None

def len_chcek(row):
      return [w for w in row if (len(w) >4) or (w == "sars") ]


Gard = pd.read_csv('/content/Gard_V1.csv')
Gard['Synonyms'] = Gard['Synonyms'].apply(lambda x: extract_words_from_json_string(x))
Gard['Synonyms'] =Gard['GardName'].apply(lambda x: [x])+Gard['Synonyms']

#######################          BOW       ########################################################################
from itertools import permutations
def generate_term_orders(terms):
    words = terms.split()
    if len(words) ==2:
      all_permutations = list(permutations(words))
      orders = [' '.join(permutation) for permutation in all_permutations]
      return orders
    else: return [terms]

def generate_term_orders_list_of_sords(words):
    X=[]
    for i in words:
      X+=generate_term_orders(i)
    return X
#Gard['Synonyms_bow']=Gard['Synonyms'].apply(lambda x: generate_term_orders_list_of_sords(x) )

########################      Removing stop words  #########################################################
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Download the stop words dataset
nltk.download('stopwords')
nltk.download('punkt')

def process_row(row):
    words = row.split()
    if len(words) > 2 :
        words = [word.lower()  for word in words if word.lower() not in ['syndrome','syndromes', 'disease','diseases']]
    return ' '.join(words)
def process_row_list(row):
      return [process_row(w) for w in row]
Gard['Synonyms_sw'] = Gard['Synonyms'].apply(lambda x: process_row_list(x))
Gard['Synonyms_sw_bow']=Gard['Synonyms_sw'].apply(lambda x: generate_term_orders_list_of_sords(x) )
Gard['Synonyms_sw_bow']=Gard['Synonyms_sw_bow'].apply(lambda x: list(set(len_chcek(x))) )

def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
def process_row_list_2(row):
    return [remove_stop_words(w) if (remove_stop_words(w) != '' and len(w.split()) > 2) else w for w in row]

#Gard['Synonyms_sw_nltk'] = Gard['Synonyms_sw'].apply(lambda x: process_row_list_2(x))
#Gard['Synonyms_sw_nltk']=Gard['Synonyms_sw_nltk']+Gard['Synonyms_sw']
#Gard['Synonyms_sw_nltk'] = Gard['Synonyms_sw_nltk'].apply(lambda x: list(set(x)))

########################      Text stemming  #########################################################
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
def stem_text(text):
    # Initialize the Porter Stemmer
    stemmer = PorterStemmer()
    # Remove punctuation
    text_without_punctuation = re.sub(r'[^\w\s]', '', text)
    # Tokenize the text into words
    words = word_tokenize(text_without_punctuation)
    # Perform stemming on each word
    stemmed_words = [stemmer.stem(word) for word in words]
    # Join the stemmed words back into a single string
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text
def stem_text_list(row):
      return [stem_text(w) for w in row if len(stem_text(w)) >2 ]

#Gard['Synonyms_stem'] = Gard['Synonyms'].apply(lambda x: stem_text_list(x))
#Gard['Synonyms_stem_bow']=Gard['Synonyms_stem'].apply(lambda x: generate_term_orders_list_of_sords(x) )
Gard['Synonyms_sw_stem'] = Gard['Synonyms_sw'].apply(lambda x: stem_text_list(x))
Gard['Synonyms_sw_stem_bow']=Gard['Synonyms_sw_stem'].apply(lambda x: generate_term_orders_list_of_sords(x) )
Gard['Synonyms_sw_stem'] = Gard['Synonyms_sw'].apply(lambda x:list(set(len_chcek(x))) )
Gard['Synonyms_sw_stem_bow']=Gard['Synonyms_sw_stem_bow'].apply(lambda x: list(set(len_chcek(x))) )

Gard['Synonyms_sw'] = Gard['Synonyms_sw'].apply(lambda x: list(set(len_chcek(x))) )

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# GardNameExtractor

In [3]:
#Gard.to_csv('Gard_V1.csv', index=False)
#Abstract = pd.read_csv('/content/Abstract_v5 (1).csv')#/content/abstract.csv')
Abstract = pd.read_csv('/content/check_.csv')#/content/abstract.csv')


In [40]:
def split_sentence(sentence):
    # Use regular expression to split words without including punctuation
    words = re.findall(r'\b\w+\b', sentence)
    return words
def word_matching(text,word):
   for i in  split_sentence(word):
     if i not in text:
        return False
   return True

def get_gard_title(text, list_chcek):
  if list_chcek in ['Synonyms_stem','Synonyms_sw_stem','Synonyms_stem_bow','Synonyms_sw_stem_bow']: text1=stem_text(text.lower())
  elif list_chcek in [ 'Synonyms_sw_nltk']  :          text1=remove_stop_words(text.lower())
  else:                                                  text1=text.lower()
  text2=split_sentence(text1)
  out=dict()
  for i in Gard.index:
    if Gard[list_chcek][i] != []:
      for j in  Gard[list_chcek][i]:
         if j in text1 and word_matching(text2,j)==True:
           if Gard['GardName'][i] in out:
                if len(j.split()) ==1:   out[Gard['GardName'][i]][0]+=text2.count(j)
                else: out[Gard['GardName'][i]][0]+=text1.count(j)
           else:
                if len(j.split()) ==1:out[Gard['GardName'][i]]=[text2.count(j)]
                else:  out[Gard['GardName'][i]]=[text1.count(j)]
  if out== {}: return None
  return out

def get_gard_title_stem_exact(text):
    exact_matching=get_gard_title(text, 'Synonyms_sw_bow')
    #print(exact_matching)
    Stemming_chcek=get_gard_title(text, 'Synonyms_sw_stem_bow')
    #print(Stemming_chcek)
    if exact_matching is None:
        exact_matching = {}
    if Stemming_chcek is None:
        Stemming_chcek = {}
    combined_dict = {}
    combined_dict.update(exact_matching)
    combined_dict.update(Stemming_chcek)
    # Remove keys that are part of another key
    keys_to_remove = set()
    for key1 in combined_dict:
        for key2 in combined_dict:
            if key1 != key2 and key1 in key2:
                keys_to_remove.add(key1)
    for key in keys_to_remove:
        del combined_dict[key]
    if combined_dict=={}:return None
    for key1 in combined_dict:
        combined_dict[key1]=1
    return combined_dict

In [ ]:
get_gard_title_stem_exact('RECOMBINANT DNA STRATEGIES--DUCHENNE MUSCULAR DYSTROPHY')

{'duchenne muscular dystrophy': [1]}

In [ ]:
Abstract1['Gard_name_title_finalized']=Abstract1.apply(lambda x: get_gard_title_stem_exact(x['project_title']), axis=1)

In [41]:
import spacy
# Load spaCy model with sentencizer component
nlp = spacy.load("en_core_web_sm")
# Function to determine verb tense
def get_verb_tense(verb):
    if "VBD" in verb.tag_:
        return "past"
    elif ("MD" in verb.tag_ and "will" in verb.lemma_.lower()) or ('aim' in verb.lemma_.lower() ) :
        return "future"
    elif "VBP" in verb.tag_ or "VBZ" in verb.tag_:
        return "present"
    else:
        return "unknown"
# Function to determine if a sentence is negated
def is_sentence_negated(sentence):
    for token in sentence:
        if token.dep_ == "neg":
            return True
    return False


def check_sen(text):
  # Process the text
  doc = nlp(text)
  # Iterate over sentences in the document
  first_sentence = ''
  Priority,Future_positive,present_positive,positive='','','',''
  for i, sent in enumerate(doc.sents, 1):
    # Initialize a set to store unique tenses in the sentence
    sentence_tenses = set()
    # Iterate over tokens in the sentence
    for token in sent:
        # Check if the token is a verb
        if token.pos == spacy.symbols.VERB or token.pos == spacy.symbols.AUX:
            # Check the tense of the verb
            tense = get_verb_tense(token)
            sentence_tenses.add(tense)

    # Determine the overall tense of the sentence
    if is_sentence_negated(sent)==False and  ("past" not in sentence_tenses):
        if i == 1:    first_sentence = sent.text
        #positive+=sent.text
        elif  ("the goal of" in sent.text.lower()) or ("aim" in sent.text.lower()):
           Priority+=sent.text
        elif "future" in sentence_tenses:
           Future_positive+=sent.text
        elif "present" in sentence_tenses and is_sentence_negated(sent)==False:
           present_positive+=sent.text
        if i == 1:    first_sentence = sent.text
  return first_sentence,Priority,Future_positive,present_positive #,
# Sample text
text = "The goal of tis project was ird. This aim is not to go the first sentence. This is not the second sentence? And this is the third sentence."
check_sen(text)

def get_sentence_with_word(paragraph, target_word):
    if not isinstance(paragraph, str):
        return ''

    # Define characters indicating the start of a new sentence
    new_sentence_chars = ['-', ':', ';', '1)', '2)', '3)', '4)', '5)', '6)', '7)', '8)']

    # Split the paragraph into sentences using provided characters
    for char in new_sentence_chars:
        paragraph = paragraph.replace(char, '.')

    # Split the paragraph into sentences using standard punctuation
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', paragraph)

    # Check for the target word in each sentence
    sen=''
    for sentence in sentences:
        if target_word.lower() in sentence.lower():
            sen+= sentence

    return sen
def stem_text_finding(text):
    # Initialize the Porter Stemmer
    stemmer = PorterStemmer()
    # Remove punctuation
    text_without_punctuation = text
    # Tokenize the text into words
    words = word_tokenize(text_without_punctuation)
    # Perform stemming on each word
    stemmed_words = [stemmer.stem(word) for word in words]
    # Join the stemmed words back into a single string
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

def split_sentence(sentence):
    # Use regular expression to split words without including punctuation
    words = re.findall(r'\b\w+\b', sentence)
    return words
def word_matching(text,word):
   for i in  split_sentence(word):
     if i not in text:
        return False
   return True

def get_gard_abstract(text, list_chcek):
  #text=check_sen(text)
  if list_chcek in ['Synonyms_stem','Synonyms_sw_stem','Synonyms_stem_bow','Synonyms_sw_stem_bow']: text1=stem_text(text.lower())
  elif list_chcek in [ 'Synonyms_sw_nltk']  :          text1=remove_stop_words(text.lower())
  else:                                                  text1=text.lower()
  text2=split_sentence(text1)
  out=dict()
  sen=dict()
  for i in Gard.index:
    if Gard[list_chcek][i] != []:
      for j in  Gard[list_chcek][i]:
         if j in text1 and word_matching(text2,j)==True:
           if Gard['GardName'][i] in out:
                if len(j.split()) ==1:
                   out[Gard['GardName'][i]][0]+=text2.count(j)
                   if list_chcek in ['Synonyms_stem','Synonyms_sw_stem','Synonyms_stem_bow','Synonyms_sw_stem_bow']:sen[Gard['GardName'][i]] += get_sentence_with_word(stem_text_finding(text.lower()), j)
                   else:    sen[Gard['GardName'][i]] += get_sentence_with_word(text1, j)
                else:
                   out[Gard['GardName'][i]][0]+=text1.count(j)
                   if list_chcek in ['Synonyms_stem','Synonyms_sw_stem','Synonyms_stem_bow','Synonyms_sw_stem_bow']:sen[Gard['GardName'][i]] += get_sentence_with_word(stem_text_finding(text.lower()), j)
                   else:    sen[Gard['GardName'][i]] += get_sentence_with_word(text1, j)
           else:
                if len(j.split()) ==1:
                     out[Gard['GardName'][i]]=[text2.count(j)]
                     if list_chcek in ['Synonyms_stem','Synonyms_sw_stem','Synonyms_stem_bow','Synonyms_sw_stem_bow']:sen[Gard['GardName'][i]] = get_sentence_with_word(stem_text_finding(text.lower()), j)
                     else:    sen[Gard['GardName'][i]] = get_sentence_with_word(text1, j)
                else:
                     out[Gard['GardName'][i]]=[text1.count(j)]
                     if list_chcek in ['Synonyms_stem','Synonyms_sw_stem','Synonyms_stem_bow','Synonyms_sw_stem_bow']:sen[Gard['GardName'][i]] = get_sentence_with_word(stem_text_finding(text.lower()), j)
                     else:    sen[Gard['GardName'][i]] = get_sentence_with_word(text1, j)
  if out== {}: return None,None
  return out,sen

In [42]:
def combine_dictionaries_count(dict1, dict2):
    combined_dict = {}
    # Update combined_dict with values from dict1
    for key, value in dict1.items():
        combined_dict[key] = combined_dict.get(key, 0) + sum(value)
    # Update combined_dict with values from dict2
    for key, value in dict2.items():
        combined_dict[key] = combined_dict.get(key, 0) + sum(value)
    return combined_dict

def combine_dictionaries_sent(dict1, dict2):
    combined_dict = {}
    # Update combined_dict with values from dict1
    for key, value in dict1.items():
        if key in combined_dict:
            combined_dict[key] += value
        else:
            combined_dict[key] = value
    # Update combined_dict with values from dict2
    for key, value in dict2.items():
        if key in combined_dict:
            combined_dict[key] += value
        else:
            combined_dict[key] = value
    return combined_dict

def  modified_dict(combined_dict,combined_dict_sen):
    keys_to_remove = set()
    for key1 in combined_dict:
        for key2 in combined_dict:
          #try:
            if key1 != key2 and (key1 in key2) and (combined_dict[key1] <= combined_dict[key2]) and (combined_dict_sen[key1] in combined_dict_sen[key2]):
                keys_to_remove.add(key1)
          #except:
          #  pass
    for key in keys_to_remove:
        del combined_dict[key]
        del combined_dict_sen[key]
    return combined_dict


def get_gard_abstract_stem_exact(text):
  if text and isinstance(text, str):
    exact_matching, exact_matching_sen=get_gard_abstract(text, 'Synonyms_sw')
    #print(exact_matching)
    Stemming_chcek, Stemming_chcek_sen=get_gard_abstract(text, 'Synonyms_sw_stem')
    #print(Stemming_chcek)
    if exact_matching is None:exact_matching = {}
    if Stemming_chcek is None:Stemming_chcek = {}
    if exact_matching_sen is None:exact_matching_sen = {}
    if Stemming_chcek_sen is None:Stemming_chcek_sen = {}

    combined_dict    = combine_dictionaries_count(exact_matching,Stemming_chcek)
    combined_dict_sen= combine_dictionaries_sent(exact_matching_sen,Stemming_chcek_sen)
    # Remove keys that are part of another key
    combined_dict=modified_dict(combined_dict,combined_dict_sen)
    if combined_dict=={}:return {}
    return combined_dict
  return {}

In [91]:
!pip install sentence-transformers torch transformers
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
import torch

def is_about_term(input_text, target_term):
    # Load ClinicalBERT model and tokenizer
    model_name = "emilyalsentzer/Bio_ClinicalBERT"
    # ClinicalBERT: "emilyalsentzer/Bio_ClinicalBERT"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    # Tokenize input text and target term
    input_tokens = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    term_tokens = tokenizer(target_term, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Get embeddings from ClinicalBERT
    with torch.no_grad():
        input_embedding = model(**input_tokens).last_hidden_state.mean(dim=1)
        term_embedding = model(**term_tokens).last_hidden_state.mean(dim=1)

    # Calculate cosine similarity between text and term
    similarity = util.pytorch_cos_sim(input_embedding, term_embedding)

    # Define a threshold for similarity
    similarity_threshold = 0.7

    # Return True if similarity is above the threshold, indicating the text is about the term
    return similarity.item() #> similarity_threshold


def harmonic_mean(numbers):
    reciprocal_sum = sum(1 / x for x in numbers)
    # Avoid division by zero
    if reciprocal_sum == 0:
        return float('inf')
    harmonic_mean = len(numbers) / reciprocal_sum
    return harmonic_mean

In [92]:
def normalize_combined_dictionary(input_text,dict1, dict2, dict3, dict4,min_, max_):
    dict1 = {key: value * 4 for key, value in dict1.items()}
    # Make the values of the second dictionary two times
    dict2 = {key: value * 3 for key, value in dict2.items()}
    dict3 = {key: value * 2 for key, value in dict3.items()}
    # Combine all dictionaries
    combined_dict = {key: dict1.get(key, 0) + dict2.get(key, 0) + dict3.get(key, 0) + dict4.get(key, 0) for key in set(dict1) | set(dict2) | set(dict3) | set(dict4)}
    # Normalize the values of the combined dictionary
    total_frequency = sum(combined_dict.values())

    # Check if total_frequency is zero to avoid division by zero
    if total_frequency == 0:
        return {}
    normalized_dict = {key: min_ + (max_ - min_) * (value / total_frequency) for key, value in combined_dict.items()}

    result_dict = {}
    for key, value in normalized_dict.items():
      if  is_about_term(input_text.lower(), key) >=0.7:
        result_dict[key] = [value, is_about_term(input_text.lower(), key)]
    return result_dict


def update_dictionary(dictionary):
    updated_dict = {}
    for key, value in dictionary.items():
        new_key = Gard[Gard['GardName'] == key]['GardId'].tolist()
        if new_key:
            new_key = new_key[0].replace('"', '')
            updated_dict[(key,new_key)] = value
        else:
            updated_dict[key] = value
    return updated_dict

def grad_id(title_, Public_health_relevance_statement, abstract_):
    if not isinstance(title_, str) and not isinstance(Public_health_relevance_statement, str) and not isinstance(abstract_, str):
        return ''  # Return default values when no string input is provided
    if title_ and isinstance(title_, str):
        name = get_gard_title_stem_exact(title_)
        if name: return name

    if Public_health_relevance_statement and isinstance(Public_health_relevance_statement, str):
        A, B, C,D = check_sen(Public_health_relevance_statement)
        name1 = get_gard_abstract_stem_exact(A)
        name2 = get_gard_abstract_stem_exact(B)
        name3 = get_gard_abstract_stem_exact(C)
        name4 = get_gard_abstract_stem_exact(D)
        name=normalize_combined_dictionary(Public_health_relevance_statement,name1,name2,name3,name4,0.7,0.9)
        if name and (name !={}): return name

    if abstract_ and isinstance(abstract_, str):
        A, B, C , D = check_sen(abstract_)
        name1 = get_gard_abstract_stem_exact(A)
        name2 = get_gard_abstract_stem_exact(B)
        name3 = get_gard_abstract_stem_exact(C)
        name4 = get_gard_abstract_stem_exact(D)
        name=normalize_combined_dictionary(abstract_,name1,name2,name3,name4,0,0.7)
        if name and (name !={}): return name


In [93]:
Abstract = pd.read_csv('/content/Sample_2.csv')#/content/abstract.csv')
Abstract1=Abstract[:100]
#Abstract1.columns
Abstract1['Gard_name']=Abstract1.apply(lambda x: grad_id(x['project_title'],x['phr_text'],x['abstract_text']), axis=1)

<ipython-input-93-b2590d54fc35>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Abstract1['Gard_name']=Abstract1.apply(lambda x: grad_id(x['project_title'],x['phr_text'],x['abstract_text']), axis=1)


In [94]:
Abstract1.to_csv('check_6.csv', index=False)

# Other methods

In [20]:
# Load pre-trained transformer model and tokenizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
model_name="bert-base-uncased"
max_seq_length=512
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_paragraph_transformer(paragraph):
    # Tokenize and encode the paragraph
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(paragraph)))
    if len(tokens) > max_seq_length:
        tokens = tokens[:max_seq_length-2]  # Account for [CLS] and [SEP]

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = torch.tensor(input_ids).unsqueeze(0)

    # Forward pass to get the transformer output
    with torch.no_grad():
        output = model(input_ids)

    # Extract the output embeddings for the entire paragraph
    embeddings = output.last_hidden_state.mean(dim=1).squeeze().numpy()

    return embeddings
#embedding = embed_paragraph_transformer(paragraph_text)
#print("Paragraph Embedding:", embedding)
# Function to calculate cosine similarity
def calculate_cosine_similarity(embedding1, embedding2):
  # Reshape the embeddings to be 2D arrays
  if embedding1 and isinstance(embedding1, str):
   if embedding2 and isinstance(embedding2, str):
    embedding1 = embed_paragraph_transformer(embedding1).reshape(1, -1)
    embedding2 = embed_paragraph_transformer(embedding2).reshape(1, -1)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)

    return similarity[0][0]
  return None

# Example usage:
paragraph_text = "This proposal is being submitted as part of a program of training of a Research Scientist Development Award.  The aim of this study is to investigate the distribution of glutamate dexarboxylase (GAD), tyrosine hydroxylase (TH), cholecystokinin (CCK), and Leu-Enkephalin (ENK) in the pre-frontal, cingulate, primary motor and visual cortex of monkey and man by means of the indirect peroxidase method for immunocytochemistry.  These markers have been chosen because they represent well-knwon transmitter and neuropeptide systems present in cerbral cortex and primary anti-sera raised against them are currently available.  Tissues will be processed for both light and electron microscopic evaluation.  At the light microscopic level, determination of cell counts of neurones in the six layers of cortex will be made by morphometric analysis.  The numbers of immunoreactive neurones for each marker will be expressed as a percentage of the total number of cells determined with Nissl-stained sections.  Electron microscopic studies for both unreacted and immunoreacted tissues will be observed for the morphological features of cell bodies positive for a given marker, and the nature of the terminals it receives.  Where two separate markers appear to interact, double-labelling immunocytochemistry will be applied.  Cerbral cortex from both schizophrenic and Huntington disease brains will also be processed for similar morphometric and immunocytochemical studies at the light, and where feasible, the electron microscopic levels.  Data from these pathological conditions associated with cognitive impairment will be compared with that obtained from control human and monkey experiments.  The overall integrity of the cortical cytoarchitecture and associated markers will be evaluated for organizational disruptions in the disease state. "
print(calculate_cosine_similarity(paragraph_text, 'arthrochalasia ehlers-danlos syndrome'))
print(calculate_cosine_similarity(paragraph_text, 'arterial tortuosity syndrome'))

0.67534995
0.46206558


In [21]:
#!pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util

def is_about_term(input_text, target_term):
    # Load pretrained BERT model
    model = SentenceTransformer('paraphrase-albert-small-v2')
    #'paraphrase-MiniLM-L3'
    # 'paraphrase-distilroberta-base-v1'
    #'paraphrase-albert-small-v2'
    #'paraphrase-MiniLM-L6-v2'
#FlagEmbedding bge-large-en-v1.5
#OpenAI text-embedding-ada-002
#HuggingFace sentence-transformers/all-MiniLM-L6-v2
#Jina AI jina-embeddings-v2-base-en
#Cohere embed-english-v3.0

    # Encode input text and target term
    text_embedding = model.encode(input_text, convert_to_tensor=True)
    term_embedding = model.encode(target_term, convert_to_tensor=True)

    # Calculate cosine similarity between text and term
    similarity = util.pytorch_cos_sim(text_embedding, term_embedding)

    # Define a threshold for similarity
    similarity_threshold = 0.6

    # Return True if similarity is above the threshold, indicating the text is about the term
    return similarity.item() > similarity_threshold

# Example usage
text_input = "The disease is characterized by rare symptoms.The disease is characterized by rare symptoms."
target_term = "rare disease"

result = is_about_term(text_input, target_term)

if result:
    print(f"The text is about the term '{target_term}'.")
else:
    print(f"The text is not about the term '{target_term}'.")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The text is about the term 'rare disease'.


In [ ]:
#ClinicalBERT
#BlueBERT
#scibert
#CamlUpper/large-scale-clinical-BERT
#

In [2]:
!pip install sentence-transformers torch transformers
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
import torch

def is_about_term(input_text, target_term):
    # Load ClinicalBERT model and tokenizer
    model_name = "emilyalsentzer/Bio_ClinicalBERT"
    # ClinicalBERT: "emilyalsentzer/Bio_ClinicalBERT"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    # Tokenize input text and target term
    input_tokens = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    term_tokens = tokenizer(target_term, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Get embeddings from ClinicalBERT
    with torch.no_grad():
        input_embedding = model(**input_tokens).last_hidden_state.mean(dim=1)
        term_embedding = model(**term_tokens).last_hidden_state.mean(dim=1)

    # Calculate cosine similarity between text and term
    similarity = util.pytorch_cos_sim(input_embedding, term_embedding)

    # Define a threshold for similarity
    similarity_threshold = 0.7

    # Return True if similarity is above the threshold, indicating the text is about the term
    return similarity.item() #> similarity_threshold

# Example usage
text_input = """
The University of Utah proposes to participate as a member of the Children's Cancer Study Group (CCSG) in the scientific design and execution of cooperative clinical research studies of childhood malignant neoplastic dieases. Utah's multi-disciplinary team of investigators have participated actively in CCSG for sixteen consecutive years. For more than a decade, Dr. Eugene Lahey led Utah's CCSG effort. In 1980, Dr. Richard O'Brien became Utah's CCSG principal investigator. He recruited a number of younger investigators with outstanding potential to supplement the experienced contributors to Utah's CCSG effort. Over the past three years Utah has aggressively enhanced its record of scientific and leadership contributions to CCSG. Dr. O'Brien, a member of the CCSG Late Effects Committee, is chairman of a proposed CCSG study (CCG-105N) expected to open for patient entry in April 1984, which will prospectively evaluate the neuropsychological effects of acute lymphoblastic leukemia (ALL) and its therapy. Dr. O'Brien is a member of a new, first line ALL study (CCG-105) and is also a member of two other new CCSG studies - Advanced Hodgkin's Disease (CCG-521) and Good Risk Medulloblastoma (CCG-923). Each of these studies address questions concerning possible late effects of therapy, an area of investigation in which Dr. O'Brien has special interest. Dr. Carl Kjeldsberg has joined Dr. John Wilson as major pathologists for CCSG lymphoma studies and they are responsible for "rapid review" of all NHL specimens. As a result, Utah serves as a Central Reference Pathology Laboratory for many of the CCSG lymphoma studies. In addition, their investigative work is defining the importance of histopathology in disease management. Dr. Dale Johnson is Vice Chairman of the Surgical Steering Committee, a member of the rhabdomyosarcoma study, a new relapsed ALL study (CCG-112) and has written the surgical guidelines for testicular biopsy for all ALL patients. Consequently, in the past three years under new leadership, there has been an impressive resurgance of Utah's scientific contributions to CCSG. Four Utah investigators are now key participants in more than half dozen new, first line CCSG studies, the results of which should be realized over the next few years. This recent record demonstrates the important role that Utah investigators play in the current and, particularly, the future scientific goals and accomplishments of CCSG."""
target_term = "rhabdomyosarcoma"

result = is_about_term(text_input, target_term)

if result:
    print(result)
    print(f"The text is about the term '{target_term}'.")
else:
    print(f"The text is not about the term '{target_term}'.")

0.7364452481269836
The text is about the term 'rhabdomyosarcoma'.
